### Training Pipeline

In [ ]:
import lightning as pl
import pandas as pd
from lightning.pytorch import loggers as pl_loggers
from lightning.pytorch.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split

from en_grammar_checker.config import Config
from en_grammar_checker.datasets import get_train_data_loader, get_val_data_loader
from en_grammar_checker.trainer import MyLightningClassifierModel

In [ ]:
cnfg = Config()

### Read Data

In [ ]:
df = pd.read_csv(
    f"{cnfg.train_path}",
    delimiter="\t",
    header=None,
    names=["sentence_source", "label", "label_notes", "sentence"],
)

#### Create Train Val DF

In [ ]:
train_df, val_df = train_test_split(
    df, test_size=cnfg.train_val_split, random_state=cnfg.seed
)
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)

In [ ]:
train_df.shape, val_df.shape

((7268, 4), (1283, 4))

#### Get DataLoaders

In [ ]:
train_dataloader = get_train_data_loader(cnfg, train_df)
val_dataloader = get_val_data_loader(cnfg, val_df)

/home/rohit/Desktop/rohit/virtualenvs/rohit_transformers/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/rohit/Desktop/rohit/virtualenvs/rohit_transformers/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


#### PL Model

In [ ]:
task = MyLightningClassifierModel(cnfg)

#### Training

In [ ]:
checkpoint_callback = ModelCheckpoint(
    # dirpath=f"{cnfg.training_logs_path}/{cnfg.experiment_name}/",
    monitor="val_f1",
    filename="{epoch}-{val_f1:.4f}",
    every_n_train_steps=1,
    mode="max",
    save_top_k=1,
)

In [ ]:
trainer = pl.Trainer(
    accelerator="gpu",
    default_root_dir=f"{cnfg.training_logs_path}/{cnfg.experiment_name}/",
    callbacks=[
        EarlyStopping(
            monitor="val_f1",
            mode="max",
            patience=cnfg.early_stopping_rounds,
        ),
        # RichProgressBar(),
        checkpoint_callback,
        # progress_bar,
    ],
    max_epochs=100,
    # enable_progress_bar=False,
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/rohit/Desktop/rohit/virtualenvs/rohit_transformers/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/logger_connector/logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default


In [ ]:
import warnings

warnings.filterwarnings("ignore")

In [ ]:
trainer.fit(task, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader)

You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
Missing logger folder: ../training_logs/test_run1/lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                | Params
----------------------------------------------
0 | model | EnDeBertaClassifier | 435 M 
----------------------------------------------
1.2 M     Trainable params
434 M     Non-trainable params
435 M     Total params
1,740.773 Total estimated model params size (MB)


Sanity Checking: |                                        | 0/? [00:00<?, ?it/s]

	 Epoch: 0, Val F1: 0.4434782608695652, Val Precision: 0.5, Val Recall 0.3984375


Training: |                                               | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

...

	 Epoch: 22, Val F1: 0.7808002517068022, Val Precision: 0.7542864414899988, Val Recall 0.8403449077413874


Validation: |                                             | 0/? [00:00<?, ?it/s]

	 Epoch: 23, Val F1: 0.8037888780562406, Val Precision: 0.7872409869445443, Val Recall 0.8291257805530776


Validation: |                                             | 0/? [00:00<?, ?it/s]

	 Epoch: 24, Val F1: 0.7950479233226837, Val Precision: 0.7719771828961552, Val Recall 0.8385875643032878


In [ ]:
# from lightning.pytorch.tuner.tuning import Tuner
# lr_finder = Tuner(trainer).lr_find(model = task,train_dataloaders=train_dataloader, val_dataloaders=val_dataloader,min_lr=1e-08, max_lr=0.1, num_training=1000)